### 6.0. Environmental Set-up & Data Loading

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
FOLDERNAME =  'ADX/'

assert FOLDERNAME is not None, 'ERROR'

%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [ ]:
data_ = pd.read_csv('./ADX/Data_CreditCard/creditcard_efsvm.csv', encoding ='cp949')
columns_ = data_.columns

In [ ]:
data_.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.004929,-0.985978,-0.038039,3.710061,-6.631951,5.122103,4.371691,-2.006868,-0.278736,-0.230873,...,1.393406,-0.381671,0.969719,0.019445,0.570923,0.333278,0.857373,-0.075538,1402.95,-1
1,1.154312,0.265462,0.384871,0.575007,-0.217475,-0.391520,-0.081489,0.062789,-0.260583,-0.161677,...,-0.193213,-0.557685,0.169492,0.186863,0.089252,0.093626,-0.009633,0.019668,2.67,-1
2,1.038370,0.127486,0.184456,1.109950,0.441699,0.945283,-0.036715,0.350995,0.118950,-0.243289,...,0.102520,0.605089,0.023092,-0.626463,0.479120,-0.166937,0.081247,0.001192,1.18,-1
3,1.287226,-0.824683,1.346423,-0.525628,-1.833007,-0.477715,-1.224213,0.014774,-0.257813,0.602162,...,-0.147633,0.057765,0.052105,0.428755,-0.036979,1.112152,-0.003678,0.031920,30.90,-1
4,-0.592060,0.785904,2.806517,2.935466,-0.356682,1.127563,-0.225130,0.514861,-0.757541,0.523216,...,-0.105161,-0.216887,-0.190514,-0.033156,-0.001445,0.200179,0.109606,0.084341,18.96,-1


### 6.1. Functions to be used

In [ ]:
def Kernel_(x, y, params = 0, type_ = 'default') :
    if type_ == 'rbf' :
      Kernel = np.exp(- (np.sum(x **2, axis = 1).reshape(-1,1) + np.sum(y **2, axis = 1).reshape(1,-1) - 2 * x @ y.T)* params)
      return Kernel
    elif type_ == 'default' :
      Kernel = np.dot(x, y.T)
      return Kernel

def Minmax_(X) :
    return (X - X.min(axis = 0)) / (X.max(axis= 0) - X.min(axis = 0)), X.max(axis =0) , X.min(axis =0)

def Standar_(X) :
    return (X - X.mean(axis =0)) / X.std(axis = 0), X.mean(axis =0), X.std(axis = 0)

def test_Standar_(X, cache): # test data의 정규화를 위해 캐시를 사용(cache hit)
  return (X - cache['mean']) / cache['std']

def Convolution(pred, real) :
    pred = np.array(pred)
    y = np.array(real)
    TP = np.sum((pred == 1) & (y == 1))
    FP = np.sum((pred == 1) & (y != 1))
    FN = np.sum((pred != 1) & (y == 1))
    TN = np.sum((pred != 1) & (y != 1))
    return TP, FP, FN, TN

## f1-score
def f1_score(X):
  TP, FP, FN, TN = X
  Precision = TP / (TP + FP)
  Recall = TP / (TP + FN)
  return (Precision * Recall) * 2 / (Precision + Recall)

############## Accuracy is newly included!
def acc_precision_recall(X) :
  TP,FP,FN,TN = X
  return (TP + TN) / (TP + FP + FN + TN), TP / (TP + FP), TP / (TP + FN)

In [ ]:
from sklearn.model_selection import train_test_split # data spliting with train, test set, respectively.

train_x, test_x, train_y, test_y = train_test_split(data_[columns_[:-1]], data_['Class'], test_size = 0.2, random_state=0)

In [ ]:
cache = {}
tr_X, cache['mean'], cache['std'] = Standar_(np.array(train_x))
tr_y = np.array(train_y) * 1.

### 6.2. Linear Kernel Support Vector Machine

In [ ]:
###### Standardization Scaler + Linear

# Hyperparameter
C = 10 

#Initializing values and computing H. Note the 1. to force to float type

# train set
tr_X = Standar_(np.array(train_x))[0]
tr_y = np.array(train_y) * 1.

# test set
t_X = test_Standar_(np.array(test_x), cache) # cache hit를 통해 test data normalize
t_y = np.array(test_y) * 1.

tr_y = tr_y.reshape(-1,1) 
m,n = tr_X.shape

#Kernel 관련 정의 numpy array 형식으로 생성
H = Kernel_(tr_X, tr_X) * 1.
H *= tr_y@tr_y.T
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(m),np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(tr_y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

w = ((tr_y * alphas).T @ tr_X).reshape(-1,1)
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = tr_y[S] - np.sum(Kernel_(tr_X, tr_X[S], type_ = 'default')* tr_y * alphas , axis = 0).reshape(-1,1)

print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('w = ', w.flatten())
print('')
print('b = ', np.mean(b))
print('')
print("support vector : ", np.array(range(m))[S])

     pcost       dcost       gap    pres   dres
 0: -1.3852e+03 -3.3325e+05  1e+06  1e+00  2e-12
 1: -1.1043e+03 -1.4959e+05  3e+05  2e-01  2e-12
 2: -8.4482e+02 -5.8984e+04  1e+05  8e-02  1e-12
 3: -7.6965e+02 -4.1854e+04  7e+04  5e-02  6e-13
 4: -7.1372e+02 -3.3043e+04  5e+04  3e-02  6e-13
 5: -6.5268e+02 -2.2977e+04  3e+04  2e-02  5e-13
 6: -6.0107e+02 -1.6055e+04  2e+04  1e-02  4e-13
 7: -5.8120e+02 -1.0010e+04  1e+04  4e-03  4e-13
 8: -5.7665e+02 -7.5510e+03  8e+03  3e-03  3e-13
 9: -5.9370e+02 -4.6041e+03  4e+03  1e-03  3e-13
10: -6.3569e+02 -4.2828e+03  4e+03  9e-04  4e-13
11: -6.7344e+02 -3.6511e+03  3e+03  6e-04  4e-13
12: -6.8956e+02 -3.1797e+03  3e+03  4e-04  4e-13
13: -7.0496e+02 -2.2329e+03  2e+03  1e-04  4e-13
14: -7.1935e+02 -2.2245e+03  2e+03  9e-05  4e-13
15: -7.0866e+02 -2.1217e+03  1e+03  6e-05  4e-13
16: -7.2717e+02 -1.8683e+03  1e+03  4e-05  4e-13
17: -7.5164e+02 -1.4269e+03  7e+02  2e-05  4e-13
18: -7.6513e+02 -1.4221e+03  7e+02  1e-05  5e-13
19: -7.7164e+02 -1.38

In [ ]:
# Solution
pred_sol = np.sign(np.sum(Kernel_(tr_X, t_X ,type_ = 'default')* tr_y * alphas, axis = 0).reshape(-1,1) + b[0])
#accuracy(Convolution(pred_sol,y))
LINEAR_STANDARD_SVM = acc_precision_recall(Convolution(pred_sol.flatten(),t_y))
acc_precision_recall(Convolution(pred_sol.flatten(),t_y)) 

(0.996, 1.0, 0.9130434782608695)

### 6.3. Gaussian (RBF) Kernel Support Vector Machines

In [ ]:
###### Standardization Scaler + RBF

# Hyperparameter
C = 10
Gamma = 2

#Initializing values and computing H. Note the 1. to force to float type

# train set
tr_X = Standar_(np.array(train_x))[0]
tr_y = np.array(train_y) * 1.

# test set
t_X = test_Standar_(np.array(test_x), cache)
t_y = np.array(test_y) * 1.

tr_y = tr_y.reshape(-1,1) 
m,n = tr_X.shape

#Kernel 관련 정의 numpy array 형식으로 생성
H = Kernel_(tr_X, tr_X, params = Gamma, type_ = 'rbf')* 1.
H *= tr_y@tr_y.T 
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(m),np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(tr_y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

#Results
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = tr_y[S] - np.sum(Kernel_(tr_X, tr_X[S]  , params = 2, type_ = 'rbf')* tr_y * alphas , axis = 0).reshape(-1,1)
print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('b = ', b)
print('')
print("support vector : ", np.array(range(m))[S])

     pcost       dcost       gap    pres   dres
 0:  2.4642e+03 -7.1560e+04  7e+04  2e-12  9e-15
 1:  1.0117e+03 -1.0887e+04  1e+04  9e-14  9e-15
 2: -2.3895e+02 -1.3030e+03  1e+03  2e-13  2e-15
 3: -3.6627e+02 -4.4467e+02  8e+01  2e-13  1e-15
 4: -3.6945e+02 -3.7439e+02  5e+00  1e-13  2e-16
 5: -3.6955e+02 -3.7021e+02  7e-01  4e-14  8e-17
 6: -3.6956e+02 -3.6969e+02  1e-01  1e-14  6e-17
 7: -3.6957e+02 -3.6959e+02  3e-02  2e-14  6e-17
 8: -3.6957e+02 -3.6958e+02  8e-03  1e-14  5e-17
 9: -3.6957e+02 -3.6957e+02  1e-03  2e-14  6e-17
10: -3.6957e+02 -3.6957e+02  4e-04  8e-15  7e-17
Optimal solution found.
Alphas =  [0.08663558 0.11274174 1.88716211 ... 0.07494454 0.08459847 0.11283827]

b =  [[-0.88716153]
 [-0.88716168]
 [-0.88716211]
 ...
 [-0.88716144]
 [-0.88716151]
 [-0.88716168]]

support vector :  [   0    1    2 ... 3997 3998 3999]


In [ ]:
# Solution
pred_sol = np.sign(np.sum(Kernel_(tr_X, t_X  , params = 2, type_ = 'rbf')* tr_y * alphas , axis = 0).reshape(-1,1) + b[0])
GAUSSIAN_STANDARD_SVM = acc_precision_recall(Convolution(pred_sol.flatten(),t_y))
acc_precision_recall(Convolution(pred_sol.flatten(),t_y))

(0.957, 1.0, 0.06521739130434782)

### 6.4. Entropy-based Fuzzy Membership & $s_i$ for EFSVM

### You should use the following hyperparameters to compute $s_i$ for EFSVM

Hyperparameter: $k = 7$, $m = 10$, $\beta$ = 1/18 $\in(0, \frac{1}{m-1}]$

In [ ]:
# You could use the following function to calculate the entropy
from scipy.stats import entropy

# You can use the following function to calculate the Euclidean distance (distance.cdist)
from scipy.spatial import distance

In [ ]:
# Hyperparameter
C = 10
Gamma = 2 
k = 7 # Number of Nearest Neighbor
m = 10
beta = 1/18

In [ ]:
#################################################################################
# TODO: Generate s_i !!!!!
# 1. Euclidean distance 계산
# 2. Negative class에 대한 Entropy 계산 (k=7 사용)
# 3. m = 10 / beta = 1/18을 사용하여 Entropy-based Fuzzy Membership 계산
# 4. Positive와 Negative class 모두에 대해 si 부여
#################################################################################

#1
def getDist(X):
    return distance.cdist(X, X, metric = 'euclidean')

#2
def getEntropy(X, y, k):
    Distance = getDist(X) # 유클리드 거리를 저장
    knn_neg = [np.argsort(Distance[idx])[1:k+1] for idx, val in enumerate(y) if val < 0] # 하나의 negative class로부터 가까운 7개의 점의 거리를 가진 점들의 인덱스를 정렬시켜서 저장한 것
    Entropy = [] # entropies
    
    for indexs in knn_neg:
        p_cnt = len([y[idx] for idx in indexs if y[idx] > 0])
        pr_pos = p_cnt / k # positive 확률
        pr_neg = 1 - pr_pos # negative 확률
        H_i = entropy([pr_pos, pr_neg]) # 엔트로피 계산
        Entropy.append(H_i) # 엔트로피 list append

    return Entropy

#3
def EFMember(Entropy, m, beta):
    H_min = min(Entropy) # minimum H
    H_max = max(Entropy) # maximum H

    sub = {}
    for l in range(1, m + 1):
        thrUp = H_min + (H_max - H_min)*(l / m)
        thrLow = H_min + (H_max - H_min)* (l - 1) / m
        
        temp = []
        for i, H in enumerate(Entropy): 
            cond = (thrLow <= H and thrUp >= H) if l == m else (thrLow <= H and thrUp > H)
            if cond: # 조건에 만족하는 엔트로피는 그 인덱스를 list에 저장
                temp.append(i)

        if len(temp) == 0: # 조건에 만족한 엔트로피가 없다면, FM계산 없이 넘어가기
          continue

        fm = 1.0 - beta * l
        sub[fm] = temp # 해당하는 점들을 FM값으로 그룹핑
   
    return sub

#4
def getsi(X, FM, y):
    si = []
    neg_class = [idx for idx, val in enumerate(y) if val < 0] # negative class들의 인덱스

    for i in range(len(y)):
      if i in neg_class: # negative class
        for j in FM:
          if neg_class.index(i) in FM[j]: # .index(i)로 참조해야 올바르게 FM[j]에서 값을 참조할 수 있음
            si.append(j)
            break
      else: # positive class
        si.append(1.0)
      
    return si

In [ ]:
# calculate entropy
entr = getEntropy(tr_X, tr_y, k)

# calculate fuzzy membership
membership = EFMember(entr, m, beta)

# calculate s_i
si_array = np.array(getsi(tr_X,membership, tr_y))

### 2.2.5. Linear Kernel Entropy Fuzzy Support Vector Machine (Standard Scaler)

In [ ]:
#################################################################################
# TODO: 위 Linear Kernel SVM을 사용하여 Linear Kernel EFSVM 구현
#################################################################################

###### Standardization Scaler + Linear

#Initializing values and computing H. Note the 1. to force to float type

# train set
tr_X = Standar_(np.array(train_x))[0]
tr_y = np.array(train_y) * 1.

# test set
t_X = test_Standar_(np.array(test_x), cache)
t_y = np.array(test_y) * 1.

tr_y = tr_y.reshape(-1,1) 
m,n = tr_X.shape

#Kernel 관련 정의 numpy array 형식으로 생성
H = Kernel_(tr_X, tr_X) * 1.
H *= tr_y@tr_y.T
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(m),np.eye(m))))

h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * si_array * C)))
A = cvxopt_matrix(tr_y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

w = ((tr_y * alphas).T @ tr_X).reshape(-1,1)
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = tr_y[S] - np.sum(Kernel_(tr_X, tr_X[S], type_ = 'default')* tr_y * alphas , axis = 0).reshape(-1,1)

print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('w = ', w.flatten())
print('')
print('b = ', np.mean(b))
print('')
print("support vector : ", np.array(range(m))[S])



     pcost       dcost       gap    pres   dres
 0: -1.3230e+03 -3.2672e+05  1e+06  1e+00  2e-12
 1: -1.0822e+03 -1.4910e+05  3e+05  3e-01  1e-12
 2: -8.3100e+02 -5.8474e+04  1e+05  8e-02  9e-13
 3: -7.4760e+02 -3.8986e+04  6e+04  4e-02  6e-13
 4: -6.6724e+02 -2.5859e+04  4e+04  2e-02  5e-13
 5: -6.1174e+02 -2.2172e+04  3e+04  2e-02  4e-13
 6: -5.6336e+02 -1.5225e+04  2e+04  7e-03  4e-13
 7: -5.5280e+02 -9.0224e+03  1e+04  3e-03  4e-13
 8: -5.6196e+02 -5.8372e+03  6e+03  1e-03  4e-13
 9: -5.9079e+02 -4.3130e+03  4e+03  8e-04  4e-13
10: -6.1445e+02 -3.8945e+03  3e+03  6e-04  4e-13
11: -6.6127e+02 -2.8891e+03  2e+03  2e-04  4e-13
12: -6.5258e+02 -2.8010e+03  2e+03  2e-04  5e-13
13: -6.7919e+02 -2.3999e+03  2e+03  1e-04  4e-13
14: -6.9520e+02 -2.2656e+03  2e+03  1e-04  4e-13
15: -6.9812e+02 -2.0141e+03  1e+03  4e-05  4e-13
16: -7.1036e+02 -1.9584e+03  1e+03  4e-05  4e-13
17: -7.0314e+02 -1.8694e+03  1e+03  2e-05  4e-13
18: -7.3842e+02 -1.5657e+03  8e+02  1e-05  4e-13
19: -7.4776e+02 -1.40

In [ ]:
# Solution
pred_sol = np.sign(np.sum(Kernel_(tr_X, t_X  ,type_ = 'default')* tr_y * alphas , axis = 0).reshape(-1,1) + b[0])
LINEAR_FUZZY_ENTROPY_SVM = acc_precision_recall(Convolution(pred_sol.flatten(),t_y))
acc_precision_recall(Convolution(pred_sol.flatten(),t_y))

(0.996, 1.0, 0.9130434782608695)

### 2.2.6. Gaussian (RBF) Kernel Entropy Fuzzy Support Vector Machine (Standard Scaler)

In [ ]:
#################################################################################
# TODO: 위 Gaussian (RBF) Kernel SVM을 사용하여 Gaussian (RBF) Kernel EFSVM 구현
#################################################################################


C = 10
Gamma = 2

#Initializing values and computing H. Note the 1. to force to float type

# train set
tr_X = Standar_(np.array(train_x))[0]
tr_y = np.array(train_y) * 1.

# test set
t_X = test_Standar_(np.array(test_x), cache)
t_y = np.array(test_y) * 1.

tr_y = tr_y.reshape(-1,1) 
m,n = tr_X.shape

#Kernel 관련 정의 numpy array 형식으로 생성
H = Kernel_(tr_X,  tr_X  , params = Gamma, type_ = 'rbf')* 1.
H *= tr_y@tr_y.T 
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(m),np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * si_array * C)))
A = cvxopt_matrix(tr_y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

#Results
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = tr_y[S] - np.sum(Kernel_(tr_X, tr_X[S]  , params = 2, type_ = 'rbf')* tr_y * alphas , axis = 0).reshape(-1,1)
print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('b = ', b[0])
print('')
print("support vector : ", np.array(range(m))[S])

     pcost       dcost       gap    pres   dres
 0:  2.3207e+03 -6.9181e+04  2e+05  3e-01  9e-15
 1:  2.6920e+03 -1.1413e+04  1e+04  7e-04  2e-15
 2:  6.9219e+01 -2.6099e+03  3e+03  2e-14  9e-15
 3: -3.4250e+02 -6.3548e+02  3e+02  1e-13  2e-15
 4: -3.6854e+02 -3.9426e+02  3e+01  2e-14  6e-16
 5: -3.6949e+02 -3.7089e+02  1e+00  9e-15  1e-16
 6: -3.6955e+02 -3.6984e+02  3e-01  2e-14  8e-17
 7: -3.6956e+02 -3.6962e+02  6e-02  8e-15  7e-17
 8: -3.6957e+02 -3.6958e+02  1e-02  1e-14  6e-17
 9: -3.6957e+02 -3.6957e+02  4e-03  1e-14  8e-17
10: -3.6957e+02 -3.6957e+02  7e-04  6e-15  7e-17
11: -3.6957e+02 -3.6957e+02  2e-04  9e-16  5e-17
Optimal solution found.
Alphas =  [0.08663554 0.11274174 1.88716187 ... 0.07494438 0.08459839 0.11283828]

b =  [-0.88716158]

support vector :  [   0    1    2 ... 3997 3998 3999]


In [ ]:
pred_sol = np.sign(np.sum(Kernel_(tr_X, t_X, params = 2, type_ = 'rbf')* tr_y * alphas , axis = 0).reshape(-1,1) + b[0])
GAUSSIAN_FUZZY_ENTROPY_SVM = acc_precision_recall(Convolution(pred_sol.flatten(),t_y))
acc_precision_recall(Convolution(pred_sol.flatten(),t_y))

(0.957, 1.0, 0.06521739130434782)

In [ ]:
LINEAR_STANDARD_SVM=pd.Series(LINEAR_STANDARD_SVM,index=['Accuracy','Precision','Recall'],name='LINEAR_STANDARD_SVM')
GAUSSIAN_STANDARD_SVM=pd.Series(GAUSSIAN_STANDARD_SVM,index=['Accuracy','Precision','Recall'],name='GAUSSIAN_STANDARD_SVM')
LINEAR_FUZZY_ENTROPY_SVM=pd.Series(LINEAR_FUZZY_ENTROPY_SVM,index=['Accuracy','Precision','Recall'],name='LINEAR_FUZZY_ENTROPY_SVM')
GAUSSIAN_FUZZY_ENTROPY_SVM=pd.Series(GAUSSIAN_FUZZY_ENTROPY_SVM,index=['Accuracy','Precision','Recall'],name='GAUSSIAN_FUZZY_ENTROPY_SVM')
table = pd.concat([LINEAR_STANDARD_SVM,GAUSSIAN_STANDARD_SVM,LINEAR_FUZZY_ENTROPY_SVM,GAUSSIAN_FUZZY_ENTROPY_SVM],1)

<ipython-input-20-decac2690782>:5: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  table = pd.concat([LINEAR_STANDARD_SVM,GAUSSIAN_STANDARD_SVM,LINEAR_FUZZY_ENTROPY_SVM,GAUSSIAN_FUZZY_ENTROPY_SVM],1)


In [ ]:
table

,LINEAR_STANDARD_SVM,GAUSSIAN_STANDARD_SVM,LINEAR_FUZZY_ENTROPY_SVM,GAUSSIAN_FUZZY_ENTROPY_SVM
Accuracy,0.996000,0.957000,0.996000,0.957000
Precision,1.000000,1.000000,1.000000,1.000000
Recall,0.913043,0.065217,0.913043,0.065217


-------------------------------------
**Question: Linear와 Gaussian (RBF) 두 Kernel에 대한 SVM과 EFSVM 결과를 Accuracy / Precision / Recall에 대해 비교하세요**


-------------------------------------

Answer: Linear, Gaussian 두 Kernel에 대한 SVM, EFSVM 결과가 정확성, 정밀도, 재현율 측면에서 모두 서로 동일함을 알 수 있다. 그리고 SVM 과 EFSVM 서로 간의 정확한 성능 차이 확인을 위해서 더 크고 다양한 dataset이 요구된다.
&nbsp;